In [1]:
import dspy
from dotenv import load_dotenv

load_dotenv()


True

In [2]:
import phoenix as px
phoenix_session = px.launch_app()

from openinference.instrumentation.dspy import DSPyInstrumentor
from opentelemetry import trace as trace_api
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

endpoint = "http://127.0.0.1:6006/v1/traces"
resource = Resource(attributes={})
tracer_provider = trace_sdk.TracerProvider(resource=resource)
span_otlp_exporter = OTLPSpanExporter(endpoint=endpoint)
tracer_provider.add_span_processor(SimpleSpanProcessor(span_exporter=span_otlp_exporter))

trace_api.set_tracer_provider(tracer_provider=tracer_provider)
DSPyInstrumentor().instrument()

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [3]:
from datasets import load_dataset

dataset = load_dataset("yqzheng/semeval2014_restaurants")
import pandas as pd
train = pd.DataFrame(dataset['train'])
test = pd.DataFrame(dataset['test'])
train.shape, test.shape

((3608, 5), (1120, 5))

In [4]:
def merge_jsons(json_list):
    result = {}
    for j in json_list:
        result.update(j)
    return result

def create_json(df):
    df['json'] = df.apply(lambda row: {row['aspect']: row['label']} , axis=1)
    return df.groupby('text')['json'].agg(merge_jsons).reset_index()

In [5]:
train_json = create_json(train)
test_json = create_json(test)
train_json.head(10)

,text,json
0,"$160 for 2 filets, 2 sides, an appetizer and d...","{'filets': 0, 'sides': 0, 'appetizer': 0, 'dri..."
1,$20 for all you can eat sushi cannot be beaten.,{'sushi': 0}
2,$20 gets you unlimited sushi of a very high qu...,"{'sushi': 1, 'sushi places': 1, 'quality': 1}"
3,"$6 and there is much tasty food, all of it fre...",{'food': 1}
4,"($200 for 2 glasses of champagne, not too expe...","{'glasses of champagne': -1, 'bottle of wine':..."
5,(Always ask the bartender for the SEASONAL bee...,"{'SEASONAL beer': 1, 'bartender': 0}"
6,(and I have eaten my share) Which impresses me...,{'serve': 1}
7,"(food was delivered by a busboy, not waiter) W...","{'food': 0, 'busboy': -1, 'waiter': -1, 'chees..."
8,- the bread at the beginning is super tasty an...,"{'bread': 1, 'pizza': 1, 'margarite pizza with..."
9,20 minutes for our reservation but it gave us ...,"{'reservation': -1, 'cocktails': 1, 'surroundi..."


In [6]:
train_dspy = [dspy.Example(review=row['text'], aspects_with_label=row['json']).with_inputs('review') for _, row in train_json.iterrows()]
test_dspy = [dspy.Example(review=row['text'], aspects_with_label=row['json']).with_inputs('review') for _, row in test_json.iterrows()]

In [7]:
from textwrap import dedent

class Review2Aspects(dspy.Signature):
    """
    Identify aspects and their sentiments from a customer review. The aspects must be words or phrases in the review.
    The response should be a Python dictionary, where each key is an aspect and the value is a sentiment label.
    A label of 1 indicates positive sentiment, 0 indicates neutral sentiment, and -1 indicates negative sentiment.
    """

    review: str = dspy.InputField(desc="a customer review")
    aspects_with_label: dict = dspy.OutputField(format=dict, desc=dedent("""
        a single Python dictionary, where each key is an aspect and the value is the label,
        with label 1 indicating positive sentiment, 0 indicating neutral sentiment, and -1
        indicating negative sentiment.
        """))

In [8]:
class ABSAPipeline(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_aspects = dspy.TypedChainOfThought(Review2Aspects)

    def forward(self, review):
        pred = self.generate_aspects(review=review)
        return dspy.Prediction(aspects_with_label=pred.aspects_with_label)

In [10]:
def validate_venn(example, pred, trace=None):
    try:
        total_count = len(merge_jsons([example.aspects_with_label, pred.aspects_with_label]).keys())
    except:
        print([example.aspects_with_label, pred.aspects_with_label])
    accurate_count = sum(1 for k, v in example.aspects_with_label.items() if pred.aspects_with_label.get(k) == v)
    if trace is None:
        return float(accurate_count) / total_count
    else:
        return accurate_count > total_count * 0.8

In [11]:
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

gpt = dspy.OpenAI(model="gpt-3.5-turbo", max_tokens=4096)
with dspy.settings.context(lm=gpt):
    uncompiled_pipeline = ABSAPipeline()
    pred = uncompiled_pipeline(test_dspy[0].review)
    optimizer = BootstrapFewShotWithRandomSearch(metric=validate_venn)
    compiled_pipeline = optimizer.compile(ABSAPipeline(), teacher=ABSAPipeline(), trainset=train_dspy[:30])
    pred = compiled_pipeline(test_dspy[0].review)
print(pred.aspects_with_label)

Going to sample between 1 and 4 traces per predictor.
Will attempt to train 16 candidate sets.


Average Metric: 5.483333333333333 / 30  (18.3): 100%|█| 30/30 [00:20<00:00,  1.45i
/Users/richy/.pyenv/versions/3.11.7/envs/llm/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:187: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)


Average Metric: 5.483333333333333 / 30  (18.3%)
Score: 18.28 for set: [0]
New best score: 18.28 for seed -3
Scores so far: [18.28]
Best score: 18.28


Average Metric: 21.75 / 30  (72.5): 100%|█████████| 30/30 [00:07<00:00,  4.00it/s]


Average Metric: 21.75 / 30  (72.5%)
Score: 72.5 for set: [16]
New best score: 72.5 for seed -2
Scores so far: [18.28, 72.5]
Best score: 72.5


...

Backing off 0.6 seconds after 3 tries calling function <function GPT3.request at 0x10dedca40> with kwargs {}
Backing off 1.0 seconds after 2 tries calling function <function GPT3.request at 0x10dedca40> with kwargs {}
Backing off 0.5 seconds after 1 tries calling function <function GPT3.request at 0x10dedca40> with kwargs {}
Backing off 1.3 seconds after 3 tries calling function <function GPT3.request at 0x10dedca40> with kwargs {}
Backing off 3.6 seconds after 3 tries calling function <function GPT3.request at 0x10dedca40> with kwargs {}


Average Metric: 16.444444444444446 / 23  (71.5):  77%|▊| 23/30 [00:21<00:06,  1.05

Backing off 0.2 seconds after 1 tries calling function <function GPT3.request at 0x10dedca40> with kwargs {'max_tokens': 2048, 'n': 1, 'temperature': 0.0}
Backing off 0.6 seconds after 1 tries calling function <function GPT3.request at 0x10dedca40> with kwargs {}
Backing off 0.3 seconds after 1 tries calling function <function GPT3.request at 0x10dedca40> with kwargs {}
Backing off 5.4 seconds after 4 tries calling function <function GPT3.request at 0x10dedca40> with kwargs {}


Average Metric: 21.944444444444446 / 30  (73.1): 100%|█| 30/30 [00:29<00:00,  1.03


Average Metric: 21.944444444444446 / 30  (73.1%)
Score: 73.15 for set: [16]
New best score: 73.15 for seed 15
Scores so far: [18.28, 72.5, 68.81, 61.43, 56.39, 55.03, 65.97, 70.22, 68.56, 70.41, 62.5, 61.51, 64.76, 66.3, 62.7, 60.56, 55.17, 65.81, 73.15]
Best score: 73.15
Average of max per entry across top 1 scores: 0.7314814814814815
Average of max per entry across top 2 scores: 0.9077777777777778
Average of max per entry across top 3 scores: 0.9466666666666667
Average of max per entry across top 5 scores: 0.975
Average of max per entry across top 8 scores: 1.0
Average of max per entry across top 9999 scores: 1.0
19 candidate programs found.
{'menu': 1, 'pub fare': 1, 'burgers': 1, 'steaks': 1, 'shepherds pie': 1, 'portabella lasagna': 1, 'vegetarians': 1}


In [11]:
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

backend = dspy.JSONBackend(model="gpt-3.5-turbo")
with dspy.settings.context(backend=backend):
    uncompiled_pipeline = ABSAPipeline()
    pred = uncompiled_pipeline(test_dspy[0].review)
    # optimizer = BootstrapFewShotWithRandomSearch(metric=validate_venn)
    # compiled_pipeline = optimizer.compile(ABSAPipeline(), teacher=ABSAPipeline(), trainset=train_dspy[:30])
    # pred = compiled_pipeline(test_dspy[0].review)
print(pred.aspects_with_label)

AttributeError: 'dict' object has no attribute 'strip'

In [48]:
backend.inspect_history(0)

===INPUT KWARGS===
{ 'messages': [ { 'content': '\n'
                             '    Definition: The output will be the aspects '
                             '(both implicit and explicit) and the aspects '
                             'sentiment polarity.\n'
                             '    \n'
                             '\n'
                             '---\n'
                             '\n'
                             'Provided the following:\n'
                             'Review: a customer review\n'
                             '\n'
                             'Please return the following fields:\n'
                             'Aspects With Label: \n'
                             '        a single Python dictionary, where each '
                             'key is an aspect and the value is the label,\n'
                             '        with label 1 indicating positive '
                             'sentiment, 0 indicating neutral sentiment, and '
             

In [ ]:
class SimpleBackend(dspy.TextBackend):
    model: str

    def _example_span(self, signature: Signature, example: Example) -> str:
        span = {}
        for name, _ in signature.fields.items():
            if name in example:
                span[name] = f"{example[name]}"

        return json.dumps(span)

        

    def prepare_request(self, signature: dspy.Signature, example: dspy.Example, config: dict, **_kwargs) -> dict:
        options = {**self.STANDARD_PARAMS, **config}

        prompt_spans = []

        # Start by getting the instructions
        prompt_spans.append(signature.instructions)

        # Generate Spans for All the demos
        for demo in example.demos:
            prompt_spans.append(self._example_span(signature, demo))

        # Generate Span for the active example
        prompt_spans.append(self._example_span(signature, example))

        content = "\n\n---\n\n".join([span.strip() for span in prompt_spans])

        messages = {"messages": [{"role": "user", "content": content}]}

        options.update(**messages)

In [54]:
Review2Aspects.fields.items()

dict_items([('review', FieldInfo(annotation=str, required=True, json_schema_extra={'desc': 'a customer review', '__dspy_field_type': 'input', 'prefix': 'Review:'})), ('aspects_with_label', FieldInfo(annotation=str, required=True, json_schema_extra={'desc': '\n        a single Python dictionary, where each key is an aspect and the value is the label,\n        with label 1 indicating positive sentiment, 0 indicating neutral sentiment, and -1\n        indicating negative sentiment.\n        ', '__dspy_field_type': 'output', 'prefix': 'Aspects With Label:'}))])